In [89]:
import numpy as np
import pandas as pd

from mne.filter import filter_data
from mne.decoding import CSP

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [90]:
# Configuration 
TRIALS_CSV = "C:/Users/ahmad/OneDrive/General/Obsidian2/Classes/Fall 2025/EE595_AppliedMachineLearning/Project/Applied-Machine-Learning-Project/Dataset/BCI Competition 2a/Trials/A09T_trials.csv"   # change to your path
SFREQ = 250.0                    # Hz sampling rate from pdf

# We are going to try focusing on only the bands we expect to change (this was mentioned in detail in V1, but I never changed it to emphasize those bands)
L_FREQ = 8.0                     
H_FREQ = 30.0

In [91]:
# Load Data
df = pd.read_csv(TRIALS_CSV)

# Restrict to one subject (Each csv has only one subject, but in the future I might concatenate all the CSVs together)
subject_id = "A09T"
df = df[df["Subject"] == subject_id]


# Exclude Time, EOG, Label, basically everything else whose columns name doesn't start with EEG
eeg_cols = [c for c in df.columns if c.startswith("EEG")]
print(f"Using {len(eeg_cols)} EEG channels:", eeg_cols)


# Get trial IDs and number of samples per trial
trial_ids = sorted(df["Trial_ID"].unique())
n_trials = len(trial_ids)
n_channels = len(eeg_cols)


# Im assuming all the trials have the same length
# For the AO1T it doees add up in the file there are 288001 rows (+1 rows cause of header) 
n_times = df[df["Trial_ID"] == trial_ids[0]].shape[0]
print(f"{n_trials} trials, {n_channels} channels, {n_times} time points per trial")

Using 22 EEG channels: ['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16']
288 trials, 22 channels, 1000 time points per trial


In [92]:
# We are creating a 3D matrix (n_trials x n_channels x n_times)
X = np.zeros((n_trials, n_channels, n_times), dtype=np.float64)
y = np.zeros(n_trials, dtype=int)

'''
For each trial, grab all rows with the same trial_id, taking only the EEG columns (n_times x n_channels)
Then transpose it to (n_channels x n_times) to be used in CSP (its how it expects it)
save it in X[i] and save the label in y[i]
'''
for i, tid in enumerate(trial_ids):
    trial = df[df["Trial_ID"] == tid]
    X[i] = trial[eeg_cols].to_numpy().T
    # same label for entire trial
    y[i] = int(trial["Label"].iloc[0])


print("X shape:", X.shape) # (n_trials, n_channels, n_times)
print("y shape:", y.shape, "classes:", np.unique(y))


X shape: (288, 22, 1000)
y shape: (288,) classes: [1 2 3 4]


In [93]:
# We want CSP to focus on the 8-30 Hz band, so remove the rest of the unneeded frequencies
for i in range(n_trials):
    X[i] = filter_data(X[i], sfreq=SFREQ, l_freq=L_FREQ, h_freq=H_FREQ, verbose=False)

In [94]:
# TODO: Try not using straisfy and split by trail instead

# We could use stratify with the previous dataset, because each row was essentialy a trial, but now with the time
# series data, we shouldn't use startisft but save entire sets of trials for each label, because we would be losing parts of the information for a trial
# which would decrease training accuracy (as its missing data it needs) and testing accuracy (it sees new data that its not familiar with or have an idea how it could possibly fit)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

When analyzing the distribution of the data of our original csv, I noticed that there was an underlying pattern, where the values of the eeg signals in all channeles for different targets, almost never overlapped, and it was very visibily the case across subjects (I only verified with 3 subjects)

So because of this, I hypothesized that although the signals vary person to person, a similiar pattern must exist for all people, which it did (verification of other subjects). This is why I instead brought my attention to dimensionality of the data (time-series) and decided to change the dataset. 

The original dataset, clearly showed a pattern, however I believed the lack of dimensioanltiy in the data, did not allow Random Forest to effecitvely uncover underlying patterns, or complex ones. At which point I attempted to use factors such as mean, variance, etc... to guide it to creating more distinctions. 

It did improve the model a bit, from 
0.2-0.33 over EEG Neuroprosthetic Dataset (about 5% better than guessing)
to
0.4 over the BCI_2a Dataset
so there was an improvement (Note that these are over two different datasets, however they contained the same channels, one just lacks time-series data (but has optimized channels) while the new one has raw time series data)


Now with the time-series data, we look at it as a matrix (channel x time)
using the MNE library, we use the CSP function (Common Spatial Patterns) that does a one vs rest protocol and assigns weights to them. 

In [95]:
# CSP FEATURE EXTRACTION

# n_components = number of CSP filters (per class one-vs-rest)
# TODO: Use grid search to find the best n_components
csp = CSP(
    n_components=8,
    reg='ledoit_wolf',
    log=True,
    # If you get an error here, you might need to downgrade or upgrade your mne library (I have it at 1.11.0)
    # You can keep your current version, but check the documentation for the right keyword
    transform_into='average_power'   
)

In [96]:
X_train_csp = csp.fit_transform(X_train, y_train)
X_test_csp = csp.transform(X_test)

print("CSP features shape (train):", X_train_csp.shape)
print("CSP features shape (test):", X_test_csp.shape)

Computing rank from data with rank=None


    Using tolerance 9e-05 (2.2e-16 eps * 22 dim * 1.8e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=1 covariance using LEDOIT_WOLF
Done.
Estimating class=2 covariance using LEDOIT_WOLF
Done.
Estimating class=3 covariance using LEDOIT_WOLF
Done.
Estimating class=4 covariance using LEDOIT_WOLF
Done.
CSP features shape (train): (230, 8)
CSP features shape (test): (58, 8)


In [97]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [98]:
# Random Forest on CSP features
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid, 
    cv=3, 
    n_jobs=-1, 
    verbose=2,
    scoring='accuracy'
)


In [99]:
print("Starting Grid Search...")
grid_search.fit(X_train_csp, y_train)

# Get Best Parameters and Model
print("/nBest Parameters found:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

best_rf = grid_search.best_estimator_

# Predict and Evaluate using the Best Model on CSP test features
y_pred = best_rf.predict(X_test_csp)

print("\n--- Test Set Results (Best Model) ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Starting Grid Search...
Fitting 3 folds for each of 216 candidates, totalling 648 fits
/nBest Parameters found: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Best Cross-Validation Score: 0.6869446343130554

--- Test Set Results (Best Model) ---
Accuracy: 0.5862068965517241
              precision    recall  f1-score   support

           1       0.57      0.87      0.68        15
           2       0.60      0.43      0.50        14
           3       0.50      0.33      0.40        15
           4       0.67      0.71      0.69        14

    accuracy                           0.59        58
   macro avg       0.58      0.59      0.57        58
weighted avg       0.58      0.59      0.57        58



In [100]:
import joblib
from sklearn.pipeline import Pipeline

'''
final_model = Pipeline([
    ('csp', csp),        # The spatial filter
    ('clf', best_rf)     # The best random forest from grid search
])

save_path = "C:/Users/ahmad/OneDrive/General/Obsidian2/Classes/Fall 2025/EE595_AppliedMachineLearning/Project/Applied-Machine-Learning-Project/Model_Training/RandomForest/BCI_Competetion/models/csp_rf_model_v2.joblib"

import os
os.makedirs(os.path.dirname(save_path), exist_ok=True)

joblib.dump(final_model, save_path)

print(f"Pipeline successfully saved")
'''

'\nfinal_model = Pipeline([\n    (\'csp\', csp),        # The spatial filter\n    (\'clf\', best_rf)     # The best random forest from grid search\n])\n\nsave_path = "C:/Users/ahmad/OneDrive/General/Obsidian2/Classes/Fall 2025/EE595_AppliedMachineLearning/Project/Applied-Machine-Learning-Project/Model_Training/RandomForest/BCI_Competetion/models/csp_rf_model_v2.joblib"\n\nimport os\nos.makedirs(os.path.dirname(save_path), exist_ok=True)\n\njoblib.dump(final_model, save_path)\n\nprint(f"Pipeline successfully saved")\n'

Trial 1:
Best Cross-Validation Score: 0.7390635680109364

--- Test Set Results (Best Model) ---
Accuracy: 0.7413793103448276
              precision    recall  f1-score   support

           1       0.82      0.60      0.69        15
           2       0.67      0.86      0.75        14
           3       0.73      0.73      0.73        15
           4       0.79      0.79      0.79        14

    accuracy                           0.74        58
   macro avg       0.75      0.74      0.74        58
weighted avg       0.75      0.74      0.74        58


Trial 2:
Best Cross-Validation Score: 0.6520277967646388

--- Test Set Results (Best Model) ---
Accuracy: 0.603448275862069
              precision    recall  f1-score   support

           1       0.47      0.47      0.47        15
           2       0.43      0.21      0.29        14
           3       0.75      1.00      0.86        15
           4       0.62      0.71      0.67        14

    accuracy                           0.60        58
   macro avg       0.57      0.60      0.57        58
weighted avg       0.57      0.60      0.57        58

Trial 3:
Best Cross-Validation Score: 0.8218842560947824

--- Test Set Results (Best Model) ---
Accuracy: 0.8103448275862069
              precision    recall  f1-score   support

           1       0.92      0.80      0.86        15
           2       1.00      0.93      0.96        14
           3       0.90      0.60      0.72        15
           4       0.59      0.93      0.72        14

    accuracy                           0.81        58
   macro avg       0.85      0.81      0.82        58
weighted avg       0.86      0.81      0.81        58

Trial 4:
Best Cross-Validation Score: 0.5303599908863066

--- Test Set Results (Best Model) ---
Accuracy: 0.5862068965517241
              precision    recall  f1-score   support

           1       0.57      0.53      0.55        15
           2       0.50      0.50      0.50        14
           3       0.69      0.60      0.64        15
           4       0.59      0.71      0.65        14

    accuracy                           0.59        58
   macro avg       0.59      0.59      0.58        58
weighted avg       0.59      0.59      0.59        58

Trial 5:
Best Cross-Validation Score: 0.45659603554340394

--- Test Set Results (Best Model) ---
Accuracy: 0.3275862068965517
              precision    recall  f1-score   support

           1       0.55      0.40      0.46        15
           2       0.30      0.21      0.25        14
           3       0.24      0.33      0.28        15
           4       0.31      0.36      0.33        14

    accuracy                           0.33        58
   macro avg       0.35      0.33      0.33        58
weighted avg       0.35      0.33      0.33        58

Trial 6:
Best Cross-Validation Score: 0.504272043745728

--- Test Set Results (Best Model) ---
Accuracy: 0.5172413793103449
              precision    recall  f1-score   support

           1       0.55      0.73      0.63        15
           2       0.43      0.21      0.29        14
           3       0.60      0.60      0.60        15
           4       0.44      0.50      0.47        14

    accuracy                           0.52        58
   macro avg       0.50      0.51      0.50        58
weighted avg       0.51      0.52      0.50        58

Trial 7:
--- Test Set Results (Best Model) ---
Accuracy: 0.7413793103448276
              precision    recall  f1-score   support

           1       0.67      0.67      0.67        15
           2       0.67      0.57      0.62        14
           3       0.85      0.73      0.79        15
           4       0.78      1.00      0.88        14

    accuracy                           0.74        58
   macro avg       0.74      0.74      0.74        58
weighted avg       0.74      0.74      0.74        58

Trial 8:
Best Cross-Validation Score: 0.799954431533379

--- Test Set Results (Best Model) ---
Accuracy: 0.7413793103448276
              precision    recall  f1-score   support

           1       0.75      0.60      0.67        15
           2       0.77      0.71      0.74        14
           3       0.77      0.67      0.71        15
           4       0.70      1.00      0.82        14

    accuracy                           0.74        58
   macro avg       0.75      0.75      0.74        58
weighted avg       0.75      0.74      0.73        58

Trial 9:
Best Cross-Validation Score: 0.6869446343130554

--- Test Set Results (Best Model) ---
Accuracy: 0.5862068965517241
              precision    recall  f1-score   support

           1       0.57      0.87      0.68        15
           2       0.60      0.43      0.50        14
           3       0.50      0.33      0.40        15
           4       0.67      0.71      0.69        14

    accuracy                           0.59        58
   macro avg       0.58      0.59      0.57        58
weighted avg       0.58      0.59      0.57        58
